In [28]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.backend_bases import key_press_handler
import numpy as np
from tkinter import *
from sklearn.linear_model import LogisticRegression
import pandas as pd
from tkinter import ttk
import tkinter
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import statsmodels.api as sm
import statsmodels.formula.api as smf
brcc_df = pd.read_csv('modified_data.csv')


In [2]:
brcc_df

,diagnosis,radius_mean,texture_mean,smoothness_mean,compactness_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,smoothness_se,compactness_se,symmetry_se,fractal_dimension_se
0,M,17.99,10.38,0.11840,0.27760,0.2419,0.07871,1.0950,0.9053,0.006399,0.04904,0.03003,0.006193
1,M,20.57,17.77,0.08474,0.07864,0.1812,0.05667,0.5435,0.7339,0.005225,0.01308,0.01389,0.003532
2,M,19.69,21.25,0.10960,0.15990,0.2069,0.05999,0.7456,0.7869,0.006150,0.04006,0.02250,0.004571
3,M,11.42,20.38,0.14250,0.28390,0.2597,0.09744,0.4956,1.1560,0.009110,0.07458,0.05963,0.009208
4,M,20.29,14.34,0.10030,0.13280,0.1809,0.05883,0.7572,0.7813,0.011490,0.02461,0.01756,0.005115
...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,M,20.92,25.09,0.10990,0.22360,0.2149,0.06879,0.9622,1.0260,0.006399,0.04310,0.02057,0.006213
552,M,21.56,22.39,0.11100,0.11590,0.1726,0.05623,1.1760,1.2560,0.010300,0.02891,0.01114,0.004239
553,M,20.13,28.25,0.09780,0.10340,0.1752,0.05533,0.7655,2.4630,0.005769,0.02423,0.01898,0.002498
554,M,16.60,28.08,0.08455,0.10230,0.1590,0.05648,0.4564,1.0750,0.005903,0.03731,0.01318,0.003892


In [3]:
cols_remove = ['compactness_mean','fractal_dimension_mean','smoothness_se']
brcc_df2 = brcc_df.drop(cols_remove, axis=1)

In [26]:
X = brcc_df2
y = brcc_df2['diagnosis']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=40)

cols = brcc_df2.columns.drop('diagnosis')
formula = 'diagnosis ~ ' + ' + '.join(cols)

logistic_fit = smf.glm(formula=formula, data=X_train, family=sm.families.Binomial()).fit()

print(logistic_fit.summary())

                        Generalized Linear Model Regression Results                         
Dep. Variable:     ['diagnosis[B]', 'diagnosis[M]']   No. Observations:                  389
Model:                                          GLM   Df Residuals:                      379
Model Family:                              Binomial   Df Model:                            9
Link Function:                                logit   Scale:                          1.0000
Method:                                        IRLS   Log-Likelihood:                -55.262
Date:                              Mon, 21 Mar 2022   Deviance:                       110.52
Time:                                      11:51:36   Pearson chi2:                     197.
No. Iterations:                                   9                                         
Covariance Type:                          nonrobust                                         
                           coef    std err          z      P>|z|      

In [5]:
predictions = logistic_fit.predict(X_test)
predictions_nominal = [ "M" if x < 0.5 else "B" for x in predictions]

In [6]:
print(classification_report(y_test, predictions_nominal, digits=3))

cfm = confusion_matrix(y_test, predictions_nominal)

true_negative = cfm[0][0]
true_positive = cfm[1][1]
accuracy_score = round((true_negative + true_positive) / len(predictions_nominal) * 100, 1)

print('Confusion Matrix: \n', cfm, '\n')
print('Accuracy Score: ', accuracy_score, '%')
cfm

              precision    recall  f1-score   support

           B      0.925     0.990     0.957       100
           M      0.983     0.881     0.929        67

    accuracy                          0.946       167
   macro avg      0.954     0.935     0.943       167
weighted avg      0.949     0.946     0.946       167

Confusion Matrix: 
 [[99  1]
 [ 8 59]] 

Accuracy Score:  94.6 %


array([[99,  1],
       [ 8, 59]], dtype=int64)

### Xây dựng UI cho Application

In [29]:
root = Tk()
root.title('Graph')
root.geometry("700x400")


def boxPlot():  
    top = Toplevel()
    top.title(combo_data.get()+' boxplot')
    
    fig = Figure(figsize=(5, 4), dpi=100)
    fig.add_subplot(111).boxplot(brcc_df[combo_data.get()])
  
    canvas = FigureCanvasTkAgg(fig, master=top)  # A tk.DrawingArea.
    canvas.draw()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

    toolbar = NavigationToolbar2Tk(canvas, top)
    toolbar.update()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)
    
#     def on_key_press(event):
#         print("you pressed {}".format(event.key))
#         key_press_handler(event, canvas, toolbar)
        
#     canvas.mpl_connect("key_press_event", on_key_press)
#     def _quit():
#         top.quit()     # stops mainloop
#         top.destroy()
#     button = tkinter.Button(master=top, text="Quit", command=_quit)
#     button.pack(side=tkinter.BOTTOM)
    
def histPlot():  
    top = Toplevel()
    top.title(combo_data.get()+' histplot')
    
    fig = Figure(figsize=(5, 4), dpi=100)
    fig.add_subplot(111).hist(brcc_df[combo_data.get()])
  
    canvas = FigureCanvasTkAgg(fig, master=top)  # A tk.DrawingArea.
    canvas.draw()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

    toolbar = NavigationToolbar2Tk(canvas, top)
    toolbar.update()
    canvas.get_tk_widget().pack(side=tkinter.TOP, fill=tkinter.BOTH, expand=1)

def heatPlot():
    corr = brcc_df.corr().round(2)
    mask = np.zeros_like(corr, dtype=bool)
    mask[np.triu_indices_from(mask)] = True

    f, ax = plt.subplots(figsize=(20, 20))

    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    sns.heatmap(corr, mask=mask, cmap=cmap, cbar_kws={"shrink": 0.5}, annot=True)

    plt.show()
def statictis():
    desc=brcc_df[combo_data.get()].describe().reset_index().to_numpy().tolist()
    for row in desc:
        my_tree_data.insert(parent="", index="end", text="", values=row)    
        
def remove_all():
    for record in my_tree_data.get_children():
        my_tree_data.delete(record)
    
def predictModel():
#     formula = 'diagnosis ~ ' + combo_class.get()

#     logistic_fit = smf.glm(formula=formula, data=X_train, family=sm.families.Binomial()).fit()

#     predictions = logistic_fit.predict(X_test)
#     predictions_nominal = [ "M" if x < 0.5 else "B" for x in predictions]
#     my_tree_class.insert(parent="", index="end", text="",values=classification_report(y_test, predictions_nominal, digits=3))
    return

# my_menu=Menu(root)
# root.config(menu=my_menu)
my_notebook = ttk.Notebook(root)
my_notebook.pack()

Data_frame = Frame(root,width=1000,height=1000)
Class_frame = Frame(root,width=1000,height=1000)

Data_frame.pack(fill='both',expand=1)
Class_frame.pack(fill='both',expand=1)

my_notebook.add(Data_frame,text='Data Analytics')
my_notebook.add(Class_frame,text='Classification')

#bảng data
lbl1 = Label(Data_frame,text='Vẽ biểu đồ cho mô hình',font=('Arial',30)).pack(pady=10)
    
option1 = list(brcc_df.columns[1:])

combo_data = ttk.Combobox(Data_frame,value=option1)
combo_data.current(0)
combo_data.pack()

box_chart_btn = Button(Data_frame,text='Create Boxplot',command=boxPlot)
box_chart_btn.pack(pady=10)

hist_chart_btn = Button(Data_frame,text='Create Histplot',command=histPlot)
hist_chart_btn.pack(pady=10)

heat_chart_btn = Button(Data_frame,text='Create Heatplot',command=heatPlot)
heat_chart_btn.pack(pady=10)

statistic_btn = Button(Data_frame,text='Show Statictis ',command=statictis)
statistic_btn.pack(pady=10)

remove_btn = Button(Data_frame,text='Remove All ',command=remove_all)
remove_btn.pack()

my_tree_data = ttk.Treeview(Data_frame)
my_tree_data.pack(fill=X)

my_tree_data['columns']=('Statictis','Value')

my_tree_data.column('#0',width=0,stretch=NO)
my_tree_data.column('Statictis',anchor=W,width=120)
my_tree_data.column('Value',anchor=W, width=120)

my_tree_data.heading('#0',text='',anchor=W)
my_tree_data.heading('Statictis',text='Statictis',anchor=CENTER)
my_tree_data.heading('Value',text='Value',anchor=CENTER)


#bảng classification

lbl2 = Label(Class_frame,text='Dự đoán mô hình',font=('Arial',30)).pack(pady=10)

option2 = list(brcc_df2.columns[1:])

combo_class = ttk.Combobox(Class_frame,value=option2)
combo_class.current(0)
combo_class.pack()

predict_model_btn = Button(Class_frame,text='Predict Model',command=predictModel)
predict_model_btn.pack(pady=10)

my_tree_class = ttk.Treeview(Class_frame)
my_tree_class.pack(fill=X)

root.mainloop()

In [ ]:
# formula = 'diagnosis ~ ' + ' texture_mean '

# logistic_fit = smf.glm(formula=formula, data=X_train, family=sm.families.Binomial()).fit()

# predictions = logistic_fit.predict(X_test)
# predictions_nominal = [ "M" if x < 0.5 else "B" for x in predictions]
# print(classification_report(y_test, predictions_nominal, digits=3))
